# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [121]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

from sklearn.ensemble import RandomForestClassifier
# !pip install lightgbm -q
import lightgbm as lgb

### 데이터 읽어오기


In [122]:
ROOT_DIR = "data"

# data
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

display(train.head())
display(test.head())

Wip Line_Dam Process Desc._Dam     Equipment_Dam Model.Suffix_Dam  \
0      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
1      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
2      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
3      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
4      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   

  Workorder_Dam  Insp. Seq No._Dam Insp Judge Code_Dam  \
0    4F1XA938-1                  1                  OK   
1    3KPM0016-2                  1                  OK   
2    4E1X9167-1                  1                  OK   
3    3K1X0057-1                  1                  OK   
4    3HPM0007-1                  1                  OK   

   CURE END POSITION X Collect Result_Dam  CURE END POSITION X Unit Time_Dam  \
0                                   240.0                                NaN   
1                                   240.0                                NaN   
2                                  1000.0                                NaN   
3                                  1000.0                                NaN   
4                                   240.0                                NaN   

   CURE END POSITION X Judge Value_Dam  ...  \
0                                  NaN  ...   
1                                  NaN  ...   
2                                  NaN  ...   
3                                  NaN  ...   
4                                  NaN  ...   

   Production Qty Collect Result_Fill2  Production Qty Unit Time_Fill2  \
0                                    7                             NaN   
1                                  185                             NaN   
2                                   10                             NaN   
3                                  268                             NaN   
4                                  121                             NaN   

   Production Qty Judge Value_Fill2  Receip No Collect Result_Fill2  \
0                               NaN                             127   
1                               NaN                               1   
2                               NaN                              73   
3                               NaN                               1   
4                               NaN                               1   

   Receip No Unit Time_Fill2  Receip No Judge Value_Fill2  \
0                        NaN                          NaN   
1                        NaN                          NaN   
2                        NaN                          NaN   
3                        NaN                          NaN   
4                        NaN                          NaN   

   WorkMode Collect Result_Fill2  WorkMode Unit Time_Fill2  \
0                              1                       NaN   
1                              0                       NaN   
2                              1                       NaN   
3                              0                       NaN   
4                              0                       NaN   

   WorkMode Judge Value_Fill2  target  
0                         NaN  Normal  
1                         NaN  Normal  
2                         NaN  Normal  
3                         NaN  Normal  
4                         NaN  Normal  

[5 rows x 464 columns]

Set ID Wip Line_Dam Process Desc._Dam  \
0  0001be084fbc4aaa9d921f39e595961b      IVI-OB6     Dam Dispenser   
1  0005bbd180064abd99e63f9ed3e1ac80      IVI-OB6     Dam Dispenser   
2  000948934c4140d883d670adcb609584      IVI-OB6     Dam Dispenser   
3  000a6bfd02874c6296dc7b2e9c5678a7      IVI-OB6     Dam Dispenser   
4  0018e78ce91343678716e2ea27a51c95      IVI-OB6     Dam Dispenser   

      Equipment_Dam Model.Suffix_Dam Workorder_Dam  Insp. Seq No._Dam  \
0  Dam dispenser #2      AJX75334501    3J1XF767-1                  1   
1  Dam dispenser #2      AJX75334501    4B1XD472-2                  1   
2  Dam dispenser #1      AJX75334501    3H1XE355-1                  1   
3  Dam dispenser #2      AJX75334501    3L1XA128-1                  1   
4  Dam dispenser #1      AJX75334501    4A1XA639-1                  1   

  Insp Judge Code_Dam  CURE END POSITION X Collect Result_Dam  \
0                  OK                                  1000.0   
1                  OK                                  1000.0   
2                  OK                                   240.0   
3                  OK                                  1000.0   
4                  OK                                   240.0   

   CURE END POSITION X Unit Time_Dam  ...  \
0                                NaN  ...   
1                                NaN  ...   
2                                NaN  ...   
3                                NaN  ...   
4                                NaN  ...   

   Production Qty Collect Result_Fill2  Production Qty Unit Time_Fill2  \
0                                  195                             NaN   
1                                   14                             NaN   
2                                   98                             NaN   
3                                   14                             NaN   
4                                    1                             NaN   

   Production Qty Judge Value_Fill2  Receip No Collect Result_Fill2  \
0                               NaN                               1   
1                               NaN                             256   
2                               NaN                               1   
3                               NaN                               0   
4                               NaN                             215   

   Receip No Unit Time_Fill2  Receip No Judge Value_Fill2  \
0                        NaN                          NaN   
1                        NaN                          NaN   
2                        NaN                          NaN   
3                        NaN                          NaN   
4                        NaN                          NaN   

   WorkMode Collect Result_Fill2  WorkMode Unit Time_Fill2  \
0                              0                       NaN   
1                              1                       NaN   
2                              0                       NaN   
3                              1                       NaN   
4                              1                       NaN   

   WorkMode Judge Value_Fill2  target  
0                         NaN     NaN  
1                         NaN     NaN  
2                         NaN     NaN  
3                         NaN     NaN  
4                         NaN     NaN  

[5 rows x 465 columns]

### 데이터 전처리


- **결측치 처리**

In [123]:
def preprocess_missing_value(train, test):
    
    x_train = train.copy()
    x_test = test.copy()
    
    print(f"""Before preprocessing missing value 
              -> train: {x_train.shape}, test: {x_test.shape}""")
    
    
    # - 모든 행이 결측치인 행 삭제 처리
    drop_cols = [col
                 for col in train.columns
                 if (train[col].notnull().sum()==0)]
    
    # - "OK"가 유입된 데이터 삭제 처리
    drop_cols.extend([
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam', 
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
        'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
        'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave', 
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
        'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
        ])
    
    # - "Workorder" 컬럼 삭제 처리
    drop_cols.extend([
        'Workorder_Dam',
        'Workorder_AutoClave',
        'Workorder_Fill1',
        'Workorder_Fill2'
    ])
    
    # - 고윳값이 1개인 컬럼 삭제 처리
    drop_cols.extend([col
                      for col in train.columns
                      if train[col].nunique() == 1])
    
    drop_cols = list(set(drop_cols))
    
    x_train = x_train.drop(columns=drop_cols)
    
    # - test 데이터에서 'target' 컬럼 삭제
    drop_cols.extend(['target'])
    x_test = x_test.drop(columns=drop_cols)
    
    print(f"""After preprocessing missing value 
              -> train: {x_train.shape}, test: {x_test.shape}""")    

    return x_train, x_test

- **이상치 처리**

In [124]:
def preprocess_outlier_value(train, test):
    
    x_train = train.copy()
    x_test = test.copy()

    print(f"""Before preprocessing outlier value 
            -> train: {x_train.shape}, test: {x_test.shape}""")

    # 이상치 제외 컬럼명
    # exception = ["COORDINATE", "PalletID", "Position", "POSITION", "WorkMode"]
    # exception_cols = [col
    #                   for col in x_train.select_dtypes(exclude='O').columns
    #                   for word in exception
    #                   if word in col]
    # outlier_cols = set(x_train.select_dtypes(exclude='O').columns)\
    #                 - set(exception_cols)

    outlier_cols = x_train.select_dtypes(exclude='O').columns

    for col in outlier_cols:
        Q1 = x_train[col].quantile(0.25)
        Q3 = x_train[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # 이상치를 상한 및 하한으로 대체
        x_train[col] = x_train[col].apply(lambda x: lower_bound if x < lower_bound else x)
        x_train[col] = x_train[col].apply(lambda x: upper_bound if x > upper_bound else x)

    print(f"""After preprocessing outlier value 
            -> train: {x_train.shape}, test: {x_test.shape}""")   
    
    return x_train, x_test

### 인코딩

In [125]:
def feature_encoding(train, test):
    categorical_cols = []

    x_train = train.copy()
    x_test = test.copy()

    print(f"""Before encoding 
            -> train: {x_train.shape}, test: {x_test.shape}""")

    categorical_df = x_train.select_dtypes(include='O')
    target = categorical_df.pop('target')

    categorical_cols = categorical_df.columns.tolist()

    # - 수치형 컬럼 중 고윳값이 5개 이하면 범주형으로 분류
    categorical_cols.extend([col 
                             for col in x_train.select_dtypes(exclude='O')
                             if x_train[col].nunique() <= 5])

    for col in categorical_cols:
        le = LabelEncoder()
        le.fit_transform(list(x_train[col].values) + list(x_test[col].values))
        
        x_train[col] = le.transform(x_train[col].values)
        x_test[col] = le.transform(x_test[col].values)

    print(f"""After encoding
            -> train: {x_train.shape}, test: {x_test.shape}""")
    print(f"""After encoding, categorical cols 
          -> train: {list(x_train.select_dtypes(include='O').columns)}, test: {list(x_test.select_dtypes(include='O').columns)}""")

    return x_train, x_test, categorical_cols

### 스케일링

In [126]:
def scaling_processing(train, test, scaler):
    
    x_train = train.copy()
    x_test = test.copy()

    print(f"""Before scaling 
            -> train: {x_train.shape}, test: {x_test.shape}""")

    # 수치형 컬럼 추출
    train_num_cols = \
        list(set(train.columns) - set(categorical_cols + ["target", "Set ID"]))
    test_num_cols = \
        list(set(train.columns) - set(categorical_cols + ["target", "Set ID"]))


    for col in train_num_cols:
        x_train[train_num_cols] = scaler.fit_transform(x_train[train_num_cols])
        x_test[test_num_cols] = scaler.transform(x_test[test_num_cols])

    print(f"""After scaling 
            -> train: {x_train.shape}, test: {x_test.shape}""")
    
    return x_train, x_test


- **저장 함수**

In [127]:
def dataframe_to_csv(df, file_name):
    df.to_csv(file_name, index = False)
    print(f'Data Saved as {file_name}')

In [128]:
# 결측치 처리
x_train, x_test = preprocess_missing_value(train, test)
# 이상치 처리
x_train, x_test = preprocess_outlier_value(x_train, x_test)
# 인코딩
x_train, x_test, categorical_cols = feature_encoding(x_train, x_test)
# 스케일링
scaler = StandardScaler()
# scaler = RobustScaler()
# scaler = Normalizer()
x_train, x_test = scaling_processing(x_train, x_test, scaler)


# 저장
dataframe_to_csv(x_train, 'preprocessed_train_data.csv')
dataframe_to_csv(x_test, 'preprocessed_test_data.csv')

Before preprocessing missing value 
              -> train: (40506, 464), test: (17361, 465)
After preprocessing missing value 
              -> train: (40506, 139), test: (17361, 139)
Before preprocessing outlier value 
            -> train: (40506, 139), test: (17361, 139)
After preprocessing outlier value 
            -> train: (40506, 139), test: (17361, 139)
Before encoding 
            -> train: (40506, 139), test: (17361, 139)
After encoding
            -> train: (40506, 139), test: (17361, 139)
After encoding, categorical cols 
          -> train: ['target'], test: ['Set ID']
Before scaling 
            -> train: (40506, 139), test: (17361, 139)
After scaling 
            -> train: (40506, 139), test: (17361, 139)
Data Saved as preprocessed_train_data.csv
Data Saved as preprocessed_test_data.csv
